In [1]:
import numpy as np
from keras.models import Sequential , Model
from keras import applications
from keras import optimizers
from keras.layers import Dense, Dropout, Activation, Flatten , GlobalAveragePooling2D
from keras.layers import Conv2D, MaxPooling2D, Lambda
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils,to_categorical
from keras.preprocessing.image import ImageDataGenerator
from keras.datasets import cifar10
from keras.optimizers import SGD
from keras.applications.inception_v3 import InceptionV3

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.cross_validation import train_test_split
from sklearn.metrics import roc_curve, auc
from sklearn.utils import shuffle
import random
import csv
import cv2
import scipy
import os
%matplotlib inline
import matplotlib.pyplot as plt

/usr/local/lib/python2.7/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/usr/local/lib/python2.7/dist-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
num_classes = 4
epochs = 20
# BASE_PATH = '/home/ec2-user/cell_classifier/'
BASE_DIR = '../'
batch_size = 32
dropout = 0.7
learning_rate = 0.0001 

In [3]:
def get_data(folder):
    X = []
    y = []

    for wbc_type in os.listdir(folder):
        if not wbc_type.startswith('.'):
#             if wbc_type in ['NEUTROPHIL', 'EOSINOPHIL']:
#                 label = 'MONONUCLEAR'
#             else:
#                 label = 'POLYNUCLEAR'
            for image_filename in os.listdir(folder + wbc_type):
                img_file = cv2.imread(folder + wbc_type + '/' + image_filename)
                img_file = scipy.misc.imresize(arr=img_file, size=(120, 160, 3))
                if img_file is not None:
                    img_arr = np.asarray(img_file)
                    X.append(img_arr)
                    y.append(wbc_type)
    X = np.asarray(X)
    y = np.asarray(y)
    return X,y

In [4]:
X_train, y_train = get_data(BASE_DIR + 'images/TRAIN/')
X_test, y_test = get_data(BASE_DIR + 'images/TEST/')
#X_test_simple, y_test_simple = get_data(BASE_DIR + 'images/TEST_SIMPLE/')


#X_train = X_train * 1./255.
#X_test = X_test * 1./255.
#X_test_simple = X_test_simple * 1./255.

/usr/local/lib/python2.7/dist-packages/scipy/misc/pilutil.py:480: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if issubdtype(ts, int):
/usr/local/lib/python2.7/dist-packages/scipy/misc/pilutil.py:483: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  elif issubdtype(type(size), float):


In [5]:
encoder = LabelEncoder()
encoder.fit(y_test)
y_train = np_utils.to_categorical(encoder.transform(y_train))
y_test = np_utils.to_categorical(encoder.transform(y_test))
#y_test_simple = np_utils.to_categorical(encoder.transform(y_test_simple))

In [6]:
def model(x_train, y_train, base_model):

    # get layers and add average pooling layer
    x = base_model.output
    x = GlobalAveragePooling2D()(x)

    # add fully-connected layer
    x = Dense(512, activation='relu')(x)

    # add output layer
    predictions = Dense(4, activation='softmax')(x)

    model = Model(inputs=base_model.input, outputs=predictions)

    # freeze pre-trained model area's layer
    for layer in base_model.layers:
        layer.trainable = False

    # update the weight that are added
    model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
    model.fit(x_train, y_train)

    # choose the layers which are updated by training
    layer_num = len(model.layers)
    for layer in model.layers[:int(layer_num * 0.9)]:
        layer.trainable = False

    for layer in model.layers[int(layer_num * 0.9):]:
        layer.trainable = True

    # update the weights
    model.compile(optimizer=SGD(lr=learning_rate, momentum=0.9), loss='categorical_crossentropy', metrics=['accuracy'])
    history = model.fit(x_train, y_train, epochs=epochs)
    return history




In [7]:
inception_model = InceptionV3(weights='imagenet', include_top=False)

## InceptionV3

In [8]:
history = model(X_train, y_train, inception_model)


Epoch 1/1
9957/9957 [==============================] - 66s - loss: 1.2315    
Epoch 1/20
9957/9957 [==============================] - 72s - loss: 0.7982 - acc: 0.6740    
Epoch 2/20
9957/9957 [==============================] - 72s - loss: 0.7056 - acc: 0.7192    
Epoch 3/20
9957/9957 [==============================] - 71s - loss: 0.6731 - acc: 0.7332    
Epoch 4/20
9957/9957 [==============================] - 73s - loss: 0.6321 - acc: 0.7533    
Epoch 5/20
9957/9957 [==============================] - 76s - loss: 0.5969 - acc: 0.7696    
Epoch 6/20
9957/9957 [==============================] - 75s - loss: 0.5799 - acc: 0.7720    
Epoch 7/20
9957/9957 [==============================] - 76s - loss: 0.5480 - acc: 0.7844    
Epoch 8/20
9957/9957 [==============================] - 79s - loss: 0.5199 - acc: 0.7954    
Epoch 9/20
9957/9957 [==============================] - 82s - loss: 0.4973 - acc: 0.8082    
Epoch 10/20
9957/9957 [==============================] - 82s - loss: 0.4846 - acc: 0.

In [9]:

print('Predicting on test data')
y_pred = np.rint(history.model.predict(X_test))
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test, y_pred))

Predicting on test data
0.5597104945717732


In [10]:
history.model.save_weights('multiclass_inception_model_epoch'+str(epochs)+'dropout'+str(dropout)+'lr'+str(learning_rate)+'.h5')

In [11]:
history.model.load_weights('multiclass_inception_model_epoch'+str(epochs)+'dropout'+str(dropout)+'lr'+str(learning_rate)+'.h5')

In [12]:
history.model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, None, None, 3) 0                                            
____________________________________________________________________________________________________
conv2d_1 (Conv2D)                (None, None, None, 32 864         input_1[0][0]                    
____________________________________________________________________________________________________
batch_normalization_1 (BatchNorm (None, None, None, 32 96          conv2d_1[0][0]                   
____________________________________________________________________________________________________
activation_1 (Activation)        (None, None, None, 32 0           batch_normalization_1[0][0]      
___________________________________________________________________________________________

In [13]:
from sklearn.metrics import confusion_matrix

y_pred_unencoded = np.argmax(y_pred, axis=1)
y_test_unencoded = np.argmax(y_test, axis=1)

print(confusion_matrix(y_test_unencoded, y_pred_unencoded))


[[398  65  14 146]
 [159 348  31  82]
 [157  18 312 133]
 [176  30  21 397]]
